In [68]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
print(torch.__version__)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

2.0.1


In [2]:
df = pd.read_csv("../data/train.csv")
df.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [82]:
np.median(df.score.values)

3.0

In [3]:
len(df)

17307

In [4]:
llm = "HuggingFaceH4/zephyr-7b-beta"

tokenizer = AutoTokenizer.from_pretrained(llm)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [5]:
tokenizer.default_chat_template


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif true == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must 

In [7]:
sample_essay = df.iloc[2].full_text.strip(" ")
sample_essay

'People always wish they had the same technology that they have seen in movies, or the best new piece of technology that is all over social media. However, nobody seems to think of the risks that these kinds of new technologies may have. Cars have been around for many decades, and now manufacturers are starting to get on the bandwagon and come up with the new and improved technology that they hope will appeal to everyone. As of right now, it seems as though the negative characteristics of these cars consume the positive idea that these manufacturers have tried to convey.\n\nCurrently, this new technology in cars has a very long way to go before being completely "driverless". Drivers still need to be on alert when they are driving, as well as control the car near any accidents or complicated traffic situations. This seems to totally defeat the purpose of the "driverless" car. Eventually the technology may improve, but nobody can be certain that the driverless car will eventually become 

In [28]:
messages = [
    {
        "role": "system",
        "content": "You are a smart essay scoring bot. Grade the score in a range of 1-6 where 1 is the worst and 6 is the best.",
    },
    {
        "role": "user", 
        "content": f"{sample_essay}"
    },
    {
        "role": "user",
        "content": "Give the score as a valid dictionary like this: `{'score': grade}`"
    }
 ]

In [29]:
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_attention_mask=False).to("cuda")

In [30]:
print(tokenizer.decode(tokenized_chat[0]))

<|system|>
You are a smart essay scoring bot. Grade the score in a range of 1-6 where 1 is the worst and 6 is the best.</s> 
<|user|>
People always wish they had the same technology that they have seen in movies, or the best new piece of technology that is all over social media. However, nobody seems to think of the risks that these kinds of new technologies may have. Cars have been around for many decades, and now manufacturers are starting to get on the bandwagon and come up with the new and improved technology that they hope will appeal to everyone. As of right now, it seems as though the negative characteristics of these cars consume the positive idea that these manufacturers have tried to convey.

Currently, this new technology in cars has a very long way to go before being completely "driverless". Drivers still need to be on alert when they are driving, as well as control the car near any accidents or complicated traffic situations. This seems to totally defeat the purpose of the

In [25]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(llm, quantization_config=quantization_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [31]:
with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
    outputs = model.generate(tokenized_chat, max_new_tokens=20, do_sample=False, pad_token_id=tokenizer.eos_token_id)

In [32]:
print(tokenizer.decode(outputs[0]))

<|system|>
You are a smart essay scoring bot. Grade the score in a range of 1-6 where 1 is the worst and 6 is the best.</s> 
<|user|>
People always wish they had the same technology that they have seen in movies, or the best new piece of technology that is all over social media. However, nobody seems to think of the risks that these kinds of new technologies may have. Cars have been around for many decades, and now manufacturers are starting to get on the bandwagon and come up with the new and improved technology that they hope will appeal to everyone. As of right now, it seems as though the negative characteristics of these cars consume the positive idea that these manufacturers have tried to convey.

Currently, this new technology in cars has a very long way to go before being completely "driverless". Drivers still need to be on alert when they are driving, as well as control the car near any accidents or complicated traffic situations. This seems to totally defeat the purpose of the

In [44]:
eval(tokenizer.decode(outputs[0]).split("<|assistant|>")[-1].split("</s>")[0].strip("\n"))

{'score': 4}

## Do it for all essays

In [46]:
def get_message(essay):
    return [
        {
            "role": "system",
            "content": "You are a smart essay scoring bot. Grade the score in a range of 1-6 where 1 is the worst and 6 is the best.",
        },
        {
            "role": "user", 
            "content": f"{essay}"
        },
        {
            "role": "user",
            "content": "Give the score as a valid dictionary like this: `{'score': grade}`"
        }
     ]

In [56]:
generations = []
scores = []

In [ ]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    essay = row.full_text.strip(" ")
    message = get_message(essay)
    inputs = tokenizer.apply_chat_template(
        message, 
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        return_attention_mask=False
    ).to("cuda")
    with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
        outputs = model.generate(
            inputs,
            max_new_tokens=20,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    outputs = tokenizer.decode(outputs[0])
    # try to parse the score
    try:
        score_dict = eval(outputs.split("<|assistant|>")[-1].split("</s>")[0].strip("\n"))
        score = int(score_dict.get("score"))
    except:
        score = 3

    generations.append(outputs)
    scores.append(score)

In [60]:
len(generations) == len(scores)

True

In [63]:
sub_df = df.loc[:len(generations)-1]

In [64]:
sub_df["generations"] = generations
sub_df["scores"] = scores

/tmp/ipykernel_51320/1577307210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["generations"] = generations
/tmp/ipykernel_51320/1577307210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["scores"] = scores


In [90]:
sub_df = sub_df.fillna(3)

In [88]:
from sklearn.metrics import cohen_kappa_score

In [107]:
cohen_kappa_score(
    list(sub_df.score.values),
    list(sub_df.scores.values.astype(int)),
    # (np.ones(len(sub_df))*6).astype(int).tolist(),
    weights="quadratic",
)

0.19447134155864432

In [117]:
def rescale_score(score, min_possible_score, max_possible_score):
    """
    Rescales a score to be between 0 and 1.

    Parameters:
    - score: The score to rescale.
    - min_possible_score: The minimum possible score.
    - max_possible_score: The maximum possible score.

    Returns:
    - Rescaled score between 0 and 1.
    """
    return (score - min_possible_score) / (max_possible_score - min_possible_score)

# For the quadratic weighted kappa, the theoretical min and max are -1 and 1 respectively.
# However, given the nature of your data, you might never reach these theoretical extremes.
# If you've calculated your score to be -0.443, you might set practical bounds based on your dataset or domain understanding.

# Assuming the practical min and max scores observed or expected in your context
min_observed_score = -1  # Adjust based on your data or domain knowledge
max_observed_score = 1   # Adjust based on your data or domain knowledge

# Rescale your kappa score
kappa_score_rescaled = rescale_score(0.19447134155864432, min_observed_score, max_observed_score)
kappa_score_rescaled

0.5972356707793222